### 1. 읽어들일 파일명 지정

In [2]:
전처리데이터셋 = '격자_count.csv'

### 2. 화면에 출력되는 칼럼명을 확인한 후, 필요한 칼럼명을 복사하여 붙여넣기

In [4]:
import pandas as pd
df = pd.read_csv(f'{전처리데이터셋}', header=0, engine='python')
전체칼럼 = df.columns.tolist()
print ("여기 칼럼명을 확인하신 후 점수계산에 필요한 칼럼명만 복사 하세요.\n", 전체칼럼)

여기 칼럼명을 확인하신 후 점수계산에 필요한 칼럼명만 복사 하세요.
 ['SPO_1KM', '서울시 병', '노인의료복', '노인주거', '장애인거주', '장애인의료', '약국', '지하철역', '버스정류장']


### 3.  "점수칼럼"에 점수로 환산할 대상인 칼럼을 수정하도록 함

In [8]:
df.columns = ['SPO_1KM', '병원개수', '노인의료', '노인주거', '장애인거주', '장애인의료', '약국', '지하철역', '버스정류장']

In [9]:
점수칼럼 =  ['병원개수', '노인의료', '노인주거', '장애인거주', '장애인의료', '약국', '지하철역', '버스정류장']
print("입력한 점수컬럼명은 다음과 같음\n", 점수칼럼)

가중치 = [1] * len(점수칼럼)

입력한 점수컬럼명은 다음과 같음
 ['병원개수', '노인의료', '노인주거', '장애인거주', '장애인의료', '약국', '지하철역', '버스정류장']


### 4. 특정 인구 및 건물 가중치 선언
- 각각의 시설과 특정 노인수에 대한 가중치를 수기로 선언함

In [10]:
#점수칼럼  순서대로 점수칼럼 개수만큼 가중치를 입력하십시오

#병원개수 = 2 -> 병원과 약국에는 폐의약품 수거함을 사용할 실 수요자가 있으므로 가중치를 2로 두었다.
#노인의료 = 1
#노인주거 = 1 
#장애인거주 = 1
#장애인의료 = 1
#약국 = 2
#지하철역 = 3 -> 현재 폐의약품 수거함의 홍보가 부족하므로 사람들이 많이 이용하는 지하철역과 버스정류장의 경우 가중치를 더 높게 두었다.
#버스정류장 = 3


가중치 = [2, 1, 1, 1, 1, 2, 3, 3]      

if (len(가중치) != len(점수칼럼)):
    print("입력한 가중치 개수가 점수칼럼 개수와 다르므로 확인후 다시 입력 바랍니다")
else:
    print("다음 줄을 실행 하세요")

다음 줄을 실행 하세요


### 5. 점수 정규화

In [11]:
pre_process_set = df.copy()
pre_process_set = pre_process_set.sort_values(by='SPO_1KM')

# Null 이 있는 칼럼은 Null을 0로 변경
pre_process_set.fillna(0, inplace=True)
##################  QGIS에서 작업한 전처리 결과 CSV 읽기 끝###################################################

################## MinMax정규화 ##########################################################################
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# 점수칼럼 정규화
norm = pre_process_set.copy()
norm.fillna(0, inplace=True)
norm.set_index('SPO_1KM', inplace=True)

norm[점수칼럼] = scaler.fit_transform(norm.loc[:,점수칼럼])
norm.reset_index(inplace=True)
################## 정규화 끝################################################################################
norm.head()

,SPO_1KM,병원개수,노인의료,노인주거,장애인거주,장애인의료,약국,지하철역,버스정류장
0,다사4252,0.018762,0.000000,0.0,0.25,0.0,0.043478,0.0,0.187500
1,다사4351,0.093809,0.166667,1.0,0.00,0.0,0.188406,0.0,0.390625
2,다사4352,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.015625
3,다사4353,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000
4,다사4451,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.031250


### 6. 총점계산

In [13]:
df = norm.copy()

for i in range(0, len(점수칼럼)):
    name = 점수칼럼[i]
    df[name] = df[name] * 가중치[i] 

df.set_index(['SPO_1KM'], inplace=True)

df['총점'] = df[점수칼럼].sum(1)
df.reset_index(inplace=True)
df.head()

,SPO_1KM,병원개수,노인의료,노인주거,장애인거주,장애인의료,약국,지하철역,버스정류장,총점
0,다사4252,0.037523,0.000000,0.0,0.25,0.0,0.086957,0.0,0.562500,0.936980
1,다사4351,0.187617,0.166667,1.0,0.00,0.0,0.376812,0.0,1.171875,2.902971
2,다사4352,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.046875,0.046875
3,다사4353,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000
4,다사4451,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.0,0.093750,0.093750


### 7. 등수 계산

In [14]:
df['순위'] = df['총점'].rank(method='dense', ascending=False).astype(int)

final = df.sort_values(by='순위')
final.reset_index(drop=True, inplace=True)

# 상위 10개만 학인
display("결과", final.head(10))

# 최종 결과 CSV 파일쓰기
final.to_csv('결과_총점.csv', index=False, sep=',', encoding='cp949')

'결과'

,SPO_1KM,병원개수,노인의료,노인주거,장애인거주,장애인의료,약국,지하철역,버스정류장,총점,순위
0,다사5844,2.000000,0.000000,0.0,0.0,0.0,2.000000,0.428571,2.062500,6.491071,1
1,다사5352,0.303940,0.000000,1.0,0.0,0.0,0.579710,1.714286,2.015625,5.613561,2
2,다사4850,0.356473,0.000000,0.0,0.0,0.0,0.637681,1.714286,2.625000,5.333440,3
3,다사5452,0.566604,0.000000,0.0,0.0,0.0,1.130435,1.285714,2.296875,5.279628,4
4,다사5351,0.382739,0.000000,0.0,0.0,0.0,0.898551,1.285714,2.531250,5.098254,5
5,다사5552,0.255159,0.000000,0.0,0.0,0.0,0.753623,2.142857,1.875000,5.026640,6
6,다사5847,1.898687,0.000000,0.0,0.0,0.0,1.594203,0.428571,0.796875,4.718336,7
7,다사5149,0.600375,0.166667,0.0,0.0,0.0,1.130435,0.000000,2.765625,4.663102,8
8,다사5752,0.315197,0.166667,0.0,0.0,0.0,0.753623,1.714286,1.687500,4.637273,9
9,다사5251,0.146341,0.666667,0.0,0.0,0.0,0.318841,0.857143,2.531250,4.520242,10
